In [1]:
# Instalar as extensões necessárias para a execução
#pip install pandas

In [2]:
#pip install fastkml

In [3]:
#pip install shapely

In [4]:
#pip install psycopg2

In [5]:
import pandas as pd
from fastkml import kml, styles
from shapely.geometry import Point

In [6]:
class Arquivo:
    def __init__(self, kml_file):
        self.arquivo = kml_file
        self.logo_url = "http://queimadas.dgi.inpe.br/queimadas/portal-static/kml/images/logo.png"
        self.legenda_url = "http://queimadas.dgi.inpe.br/queimadas/portal-static/kml/images/legend.png"

    def iniciaArquivo(self):
        self.arquivo.write('<?xml version="1.0" encoding="UTF-8"?>')
    
    # Funções abaixo necessárias pois a biblioteca FastKML não possui a propriedade "ScreenOverlay"    
    def montarLogo(self):
        logo = f"""
    <ScreenOverlay>
      <name>Logo</name>
      <Icon>
        <href>{self.logo_url}</href>
      </Icon>
      <overlayXY x="1" y="1" xunits="fraction" yunits="fraction"/>
      <screenXY x="0.99" y="0.7" xunits="fraction" yunits="fraction"/>
      <size x="0" y="0" xunits="fraction" yunits="fraction"/>
    </ScreenOverlay>"""
        self.arquivo.writelines(logo)

    def montarLegenda(self):
        legenda = f"""
    <ScreenOverlay>
      <name>Legenda</name>
      <Icon>
        <href>{self.legenda_url}</href>
      </Icon>
      <overlayXY x="1" y="1" xunits="fraction" yunits="fraction"/>
      <screenXY x="0.99" y="0.55" xunits="fraction" yunits="fraction"/>
      <size x="0" y="0" xunits="fraction" yunits="fraction"/>
    </ScreenOverlay>"""
        self.arquivo.writelines(legenda)
        
    def finalizarArquivo(self):
        fim = """
  </Document>
</kml>
        """
        self.arquivo.writelines(fim)

In [7]:
k = kml.KML()
ns = '{http://www.opengis.net/kml/2.2}'
# Lista de satélites e cores correspondentes
SATELITES = ['AQUA_M-M', 'AQUA_M-T', 'GOES-16', 'METOP-B', 'METOP-C', 'MSG-03',
            'NOAA-18', 'NOAA-19', 'NOAA-20', 'NPP-375', 'TERRA_M-M', 'TERRA_M-T']
CORES = ('ff00ff00', 'ff00ff00', 'ffff1d00', 'ff14f0ff', 'ffcc3cf0', 'ffffff00',
            'ff0099ff', 'ff003399', 'ff6666ff', 'FFda01C1', 'ffccffcc', 'ffccffcc')

# Cria estilos e os adiciona numa lista
estilos = []
for i in range(len(SATELITES)):
    icon_style = styles.IconStyle(icon_href='http://maps.google.com/mapfiles/kml/shapes/placemark_square.png', scale=0.8, color=CORES[i])
    label_style = styles.LabelStyle(ns=ns, scale=0.6)
    lista_styles = styles.Style(styles = [icon_style, label_style], id=SATELITES[i])
    estilos.append(lista_styles)

In [8]:
# Cria o documento e adiciona a lista de estilos a ele
documento = kml.Document(ns=ns, name='Monitoramento de Queimadas Teste', description='Teste Pandas/SQLAlchemy', styles=estilos)
k.append(documento)

In [9]:
file = r'csv\Focos_2021-11-01_2021-11-03.csv'

df = pd.read_csv(file)
df

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2021/11/01 04:10:00,NOAA-20,Brasil,GOIAS,SAO SIMAO,Mata Atlantica,0.0,0.1,0.1,-19.05603,-50.59894,1.0
1,2021/11/01 16:41:00,NOAA-20,Brasil,GOIAS,SAO MIGUEL DO ARAGUAIA,Cerrado,1.0,5.9,0.3,-13.45615,-50.52847,1.6
2,2021/11/01 16:41:00,NOAA-20,Brasil,GOIAS,FORMOSA,Cerrado,1.0,0.8,0.8,-15.05474,-47.51883,12.2
3,2021/11/01 16:41:00,NOAA-20,Brasil,GOIAS,FORMOSA,Cerrado,1.0,0.8,0.6,-15.05527,-47.52240,21.9
4,2021/11/01 16:41:00,NOAA-20,Brasil,GOIAS,FORMOSA,Cerrado,1.0,0.8,0.9,-15.05806,-47.51831,12.2
...,...,...,...,...,...,...,...,...,...,...,...,...
145,2021/11/03 16:03:00,NOAA-20,Brasil,GOIAS,CUMARI,Cerrado,NaN,NaN,NaN,-18.22014,-48.20645,10.4
146,2021/11/03 16:03:00,NOAA-20,Brasil,GOIAS,CATALAO,Cerrado,NaN,NaN,NaN,-18.06046,-47.38191,4.3
147,2021/11/03 16:50:00,AQUA_M-T,Brasil,GOIAS,CUMARI,Cerrado,NaN,NaN,NaN,-18.22040,-48.20864,7.5
148,2021/11/03 18:03:49,GOES-16,Brasil,GOIAS,FORMOSA,Cerrado,NaN,NaN,NaN,-15.41000,-47.29000,NaN


In [10]:
df_filtered = df.loc[:, ['latitude', 'longitude', 'datahora', 'satelite', 'estado', 'municipio']]
df_filtered

,latitude,longitude,datahora,satelite,estado,municipio
0,-19.05603,-50.59894,2021/11/01 04:10:00,NOAA-20,GOIAS,SAO SIMAO
1,-13.45615,-50.52847,2021/11/01 16:41:00,NOAA-20,GOIAS,SAO MIGUEL DO ARAGUAIA
2,-15.05474,-47.51883,2021/11/01 16:41:00,NOAA-20,GOIAS,FORMOSA
3,-15.05527,-47.52240,2021/11/01 16:41:00,NOAA-20,GOIAS,FORMOSA
4,-15.05806,-47.51831,2021/11/01 16:41:00,NOAA-20,GOIAS,FORMOSA
...,...,...,...,...,...,...
145,-18.22014,-48.20645,2021/11/03 16:03:00,NOAA-20,GOIAS,CUMARI
146,-18.06046,-47.38191,2021/11/03 16:03:00,NOAA-20,GOIAS,CATALAO
147,-18.22040,-48.20864,2021/11/03 16:50:00,AQUA_M-T,GOIAS,CUMARI
148,-15.41000,-47.29000,2021/11/03 18:03:49,GOES-16,GOIAS,FORMOSA


In [11]:
# Cria a Folder principal
folder = kml.Folder(ns=ns, name='Focos por Satélite', description='Focos do último dia categorizado pelo satélite que o captou')
documento.append(folder)

In [12]:
subfolders_names = []
subfolders_objects = []
for foco in df_filtered.itertuples():
    # Para organizar os focos em pastas, e não recriá-las
    if not foco.satelite in subfolders_names:
        subFolder = kml.Folder(ns=ns, name=foco.satelite)
        folder.append(subFolder)
        subfolders_names.append(foco.satelite)
        subfolders_objects.append(subFolder)
        
    descricao = f"""LAT = {foco.latitude}
    LONG = {foco.longitude}
    DATA = {foco.datahora}
    SATÉLITE = {foco.satelite}
    ESTADO = {foco.estado}
    MUNICÍPIO = {foco.municipio}
    """

    ponto = kml.Placemark(ns=ns, name=foco.satelite, description=descricao, styleUrl=foco.satelite)
    ponto.geometry = Point([float(foco.longitude), float(foco.latitude)])
    subfolders_objects[subfolders_names.index(foco.satelite)].append(ponto)

In [13]:
with open(r'kml_pandas_csv.kml', 'w+') as kml_file:
    arquivo = Arquivo(kml_file)
    arquivo.iniciaArquivo()
    # slicing de -22 para remover as propriedades </Document> e </kml>
    # para tornar possível a adição de Screens Overlays
    kml_file.writelines(k.to_string(prettyprint=True)[:-22])
    arquivo.montarLogo()
    arquivo.montarLegenda()
    arquivo.finalizarArquivo()